In [1]:
from collections import defaultdict
import pandas as pd

In [2]:
# extract data
with open("07_input.txt") as f:
    lines = [l.strip() for l in f.readlines()]

# store as dict
data = [{"hand": x.split(" ")[0], "bid": int(x.split(" ")[1])} for x in lines]
data[0:5]

[{'hand': '8T64Q', 'bid': 595},
 {'hand': '79J27', 'bid': 258},
 {'hand': '88885', 'bid': 88},
 {'hand': '8933J', 'bid': 444},
 {'hand': '72527', 'bid': 676}]

In [3]:
# function to return points of hand
def check_hand(hand):
    # get letters
    values = [i for i in hand]
    value_counts = defaultdict(lambda:0)
    
    # store counts per letter
    for v in values:
        value_counts[v]+=1
    
    # sort the counts
    vals = sorted(value_counts.values())
    
    # check for the hand results
    if vals == [5]:
        return 7 # five of a kind
    elif vals == [1,4]:
        return 6 # four of a kind
    elif vals == [2,3]:
        return 5 # full house
    elif vals == [1,1,3]:
        return 4 # three of a kind
    elif vals == [1,2,2]:
        return 3 # two pairs
    elif vals == [1,1,1,2]:
        return 2 # one pair
    else:
        return 1 # high card

In [4]:
# init values for sorting
values = {"2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "T":10, "J":11, "Q":12, "K":13, "A":14}
new_data = []

# for each hand
for x in data:
    # get the points
    pts = check_hand(x["hand"])
    
    # create the row with tie breakers
    new_data.append({"hand": x["hand"], 
                     "bid": x["bid"], 
                     "points": pts,
                     "tie1": sum([values[y] for y in x["hand"][0:1]]),
                     "tie2": sum([values[y] for y in x["hand"][0:2]]),
                     "tie3": sum([values[y] for y in x["hand"][0:3]]),
                     "tie4": sum([values[y] for y in x["hand"][0:4]]),
                     "tie5": sum([values[y] for y in x["hand"]])
                    })

# create dataframe and sort
df = pd.DataFrame(new_data)
df = df.sort_values(by=["points", "tie1", "tie2", "tie3", "tie4", "tie5"], ascending=True).reset_index(drop=True)

# calculate rank and winnings
df["rank"] = df.index + 1
df["winnings"] = df.bid * df["rank"]
df.head()

,hand,bid,points,tie1,tie2,tie3,tie4,tie5,rank,winnings
0,2346J,689,1,2,5,9,15,26,1,689
1,235KQ,966,1,2,5,10,23,35,2,1932
2,23859,347,1,2,5,13,18,27,3,1041
3,23A4J,18,1,2,5,19,23,34,4,72
4,2479Q,227,1,2,6,13,22,34,5,1135


In [5]:
print(f"Part 1: {df.winnings.sum()}")

Part 1: 250474325


In [6]:
# Part 2 - Jokers
values_J = {"J":1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "T":10, "Q":11, "K":12, "A":13}
new_data_J = []

# for each hand
for x in data:
    # try all other values for J, and keep track of points found
    pts_list = []
    for r in '23456789TQKA':
        pts_list.append(check_hand(x["hand"].replace("J", r)))
    
    # use max points
    pts = max(pts_list)
    
    # create new row with tie breakers
    new_data_J.append({"hand": x["hand"], 
                     "bid": x["bid"], 
                     "points": pts,
                     "tie1": sum([values_J[y] for y in x["hand"][0:1]]),
                     "tie2": sum([values_J[y] for y in x["hand"][0:2]]),
                     "tie3": sum([values_J[y] for y in x["hand"][0:3]]),
                     "tie4": sum([values_J[y] for y in x["hand"][0:4]]),
                     "tie5": sum([values_J[y] for y in x["hand"]])
                    })

# dataframe and sort
df_J = pd.DataFrame(new_data_J)
df_J = df_J.sort_values(by=["points", "tie1", "tie2", "tie3", "tie4", "tie5"], ascending=True).reset_index(drop=True)

# calculate rank and winnings
df_J["rank"] = df_J.index + 1
df_J["winnings"] = df_J.bid * df_J["rank"]
df_J.head()

,hand,bid,points,tie1,tie2,tie3,tie4,tie5,rank,winnings
0,235KQ,966,1,2,5,10,22,33,1,966
1,23859,347,1,2,5,13,18,27,2,694
2,2479Q,227,1,2,6,13,22,33,3,681
3,2479K,955,1,2,6,13,22,34,4,3820
4,257A6,695,1,2,7,14,27,33,5,3475


In [7]:
print(f"Part 2: {df_J.winnings.sum()}")

Part 2: 248909434
